In [3]:
import cobra
import pandas as pd
import numpy as np
import os
import sys

In [22]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [24]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/gangl/Documents/GitHub/Halo-GEM/ecpy/utils.py'>

In [6]:
model = cobra.io.load_json_model('../../../ModelFiles/json/Halo_GEM_v1.json')

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only


#### 1. Add PHA synthesis reaction
(R)-3-Hydroxybutanoyl-CoA  -> CoA + PHA  
 

|Name                     |MetaCyc ID |
|:------------------------|-----------|
|(R)-3-Hydroxybutanoyl-CoA|CPD-650    |
|CoA                      |CO-A       |

The reactions is catalyzed by phaC gene. It has two isoforms: **TD01GL001170**, **TD01GL001573**. The first gene palys the major rule in PHA synthesis

In [7]:
met_cpd = model.metabolites.get_by_id('CPD-650_c')

In [8]:
for rxn in met_cpd.reactions: print(rxn.id, rxn.reaction,rxn.gene_reaction_rule)

RXN-5901 ACETOACETYL-COA_c + NADPH_c + PROTON_c --> CPD-650_c + NADP_c TD01GL000448 or TD01GL001906
RXN-14255 CPD-650_c + WATER_c --> CO-A_c + CPD-335_c + PROTON_c TD01GL003102
3-HYDROXBUTYRYL-COA-DEHYDRATASE-RXN CPD-650_c --> CROTONYL-COA_c + WATER_c TD01GL003540


In [9]:
rxn_pha = cobra.Reaction('PHA_synthetase',name='Poly(3-hydroxyalkanoate) synthetase')
met_pha = cobra.Metabolite('PHA_c',name='PHA',compartment='c')
rxn_pha.add_metabolites({
                            met_cpd                              :-1,
                            model.metabolites.get_by_id('CO-A_c'): 1,
                            met_pha                              : 1,
                        })
rxn_pha.annotation = {'ec-code':'2.3.1-'} # from kegg annotation 
rxn_pha.gene_reaction_rule = 'TD01GL001170 or TD01GL001573'

rxn_pha_sec = cobra.Reaction('PHA_secretion',name='PHA secretion')
rxn_pha_sec.add_metabolites({met_pha:-1})

In [10]:
print(rxn_pha.reaction)
print(rxn_pha_sec.reaction)

CPD-650_c --> CO-A_c + PHA_c
PHA_c --> 


In [11]:
model.add_reactions([rxn_pha,rxn_pha_sec])

In [12]:
model.objective='PHA_secretion'
model.objective_direction = 'max'

In [13]:
model.optimize()

,fluxes,reduced_costs
CHOLD,0.000000,0.000000
DEOXYRIBOSE-P-ALD-RXN,7.500000,0.000000
RXN-6321,0.000000,0.000000
GLYOHMETRANS-RXN,-106.964286,-0.000000
LTAA-RXN,0.000000,0.000000
...,...,...
NADS1,0.000000,-1.714286
THZPSN3,0.000000,-0.535714
Biomass_v1,0.000000,-37.389430
PHA_synthetase,236.964286,0.000000


In [14]:
## add gene name
utils.correct_genename(model,'TD01GL001170','phaC1')
utils.correct_genename(model,'TD01GL001573','phaC2')

#### 2. Curate gene reaction rules of reactions involved in PHA biosynthesis

##### 2.1 phaB
TD01GL000448 has been experimentally confirmed.

In [15]:
# phaB
utils.print_reactions_of_gene(model,'TD01GL000448')
utils.correct_genename(model,'TD01GL000448','phaB1')

TD01GL000448 or TD01GL001906
acetoacetyl-CoA + NADPH + H+ --> (3R)-3-hydroxybutanoyl-CoA + NADP+

TD01GL000448
(3R)-3-hydroxypentanoyl-CoA + NADP+ --> 3-oxopentanoyl-CoA + NADPH + H+



##### 2.2 phaA
There are 5 suspected phaA genes, in which TD01GL000367 (phaA3) has been experimentally confirmed. In the model, there are four genes (phaA3,4,5 and TD01GL001328) catalyze this reaction, we keep it as it is. phaA1 and phaA2 in the model catalyze other reactions. A new phaA gene was predicted

In [16]:
# phaA3, only correct gene name for this one
utils.print_reactions_of_gene(model,'TD01GL000367')
utils.correct_genename(model,'TD01GL000367','phaA3')

TD01GL000367 or TD01GL001318 or TD01GL001328 or TD01GL001300
2.0 acetyl-CoA <=> acetoacetyl-CoA + coenzyme A



In [17]:
# phaA4
utils.print_reactions_of_gene(model,'TD01GL001300')

TD01GL000367 or TD01GL001318 or TD01GL001328 or TD01GL001300
2.0 acetyl-CoA <=> acetoacetyl-CoA + coenzyme A



In [18]:
# phaA5
utils.print_reactions_of_gene(model,'TD01GL001318')

TD01GL000367 or TD01GL001318 or TD01GL001328 or TD01GL001300
2.0 acetyl-CoA <=> acetoacetyl-CoA + coenzyme A

TD01GL001318 or TD01GL001328
acetyl-CoA + butanoyl-CoA <=> coenzyme A + 3-oxohexanoyl-CoA

TD01GL001318
coenzyme A + 3-oxopentanoyl-CoA --> acetyl-CoA + propanoyl-CoA



In [19]:
# phaA1
utils.print_reactions_of_gene(model,'TD01GL003533')

TD01GL003533
3-oxoadipyl-CoA + coenzyme A --> acetyl-CoA + succinyl-CoA

TD01GL003530 and TD01GL003533
coenzyme A + 3-oxo-5,6-didehydrosuberyl-CoA --> 2,3-dehydroadipyl-CoA + acetyl-CoA

TD01GL003533
coenzyme A + 3-oxo-5,6-didehydrosuberyl-CoA --> acetyl-CoA + trans-2,3-dehydroadipyl-coA



In [20]:
# phaA2
utils.print_reactions_of_gene(model,'TD01GL000275')

TD01GL000275
coenzyme A + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-(3-oxobutanoyl)-CoA --> acetyl-CoA + (+)-7-epi-jasmonoyl-CoA

TD01GL000275
coenzyme A + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-(3-oxohexanoyl)-CoA --> acetyl-CoA + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-butanoyl-CoA

TD01GL000275
coenzyme A + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-(3-oxooctanoyl)-CoA --> acetyl-CoA + 3-oxo-2-(cis-2'-pentenyl)-cyclopentane-1-hexanoyl-CoA



#### 3. update model files

In [26]:
utils.save_model_into_formats(model,'../../../ModelFiles','Halo_GEM_v1')

Saved ../../../ModelFiles/xml/Halo_GEM_v1.xml
Saved ../../../ModelFiles/mat/Halo_GEM_v1.mat
Saved ../../../ModelFiles/json/Halo_GEM_v1.json

